# TNT Sticky Rouse: Flow Curve Fitting
> **Handbook:** This notebook demonstrates the TNT Sticky Rouse model. For complete mathematical derivations and theoretical background, see [TNT Sticky Rouse Documentation](../../docs/source/models/tnt/tnt_sticky_rouse.rst).


**Estimated Time:** 3-5 minutes

## Welcome to TNT Sticky Rouse (Hierarchical Sticker Dynamics)

This notebook is the **gateway** to the TNT Sticky Rouse model — combining **Rouse dynamics** (unentangled polymer chain modes) with **sticky associations** (reversible stickers along the backbone).

### What is Sticky Rouse?

Polymers with multiple stickers that transiently associate, where:
- **Rouse backbone**: Chain relaxes via N normal modes with $\tau_{R,k} \sim 1/k^2$ scaling
- **Sticky associations**: Stickers bind/unbind with lifetime $\tau_s$
- **Effective timescales**: $\tau_{eff,k} = \tau_{R,k} + \tau_s$ (Rouse + sticky contribution)

Materials: Multi-block copolymers, telechelic associating polymers, vitrimers in Rouse regime.

### Sticky Rouse Signatures

| Feature | Prediction | Physical Origin |
|---------|------------|-----------------|
| **SAOS** | $G'(\omega) \sim \omega^{1/2}$ at intermediate $\omega$ | Rouse scaling in sticky regime |
| **Relaxation** | Power-law + exponential tail | Hierarchical mode spectrum |
| **Flow curve** | Multi-stage shear thinning | Modes relax sequentially |

### Mathematical Framework

Each Rouse mode k has:

$$
\tau_{eff,k} = \tau_{R,k} + \tau_s = \frac{\tau_{R,1}}{k^2} + \tau_s
$$

Total stress = sum over N modes. At high frequency ($\omega \tau_s \gg 1$), Rouse scaling emerges.

> **Sticky Rouse Handbook**  
> [../../docs/source/models/tnt/tnt_sticky_rouse.rst](../../docs/source/models/tnt/tnt_sticky_rouse.rst)

---

## Learning Objectives

1. Understand Rouse + sticky dynamics
2. Fit sticky Rouse to flow curve
3. Extract $\tau_{R,1}$, $\tau_s$, N modes
4. Analyze hierarchical relaxation
5. Predict Rouse scaling regime

## Runtime: ~10-20 min

## Setup

In [ ]:
import os
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import matplotlib.pyplot as plt
import numpy as np

from rheojax.core.jax_config import safe_import_jax

jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64

verify_float64()

from rheojax.models.tnt import TNTStickyRouse

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    compute_fit_quality,
    get_tnt_sticky_rouse_param_names,
    load_ml_ikh_flow_curve,
    plot_mode_decomposition,
    plot_sticky_rouse_effective_times,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
)
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

print("Setup complete. JAX devices:", jax.devices())

# Residual analysis
residuals = stress - model.predict(gamma_dot, test_mode="flow_curve")
print(f"\nResidual Statistics:")
print(f"  Mean residual = {np.mean(residuals):.4e}")
print(f"  Std residual = {np.std(residuals):.4e}")
print(f"  Max absolute residual = {np.max(np.abs(residuals)):.4e}")


### Bayesian Convergence Diagnostics

When running full Bayesian inference (FAST_MODE=0), monitor these diagnostic metrics to ensure MCMC convergence:

| Metric | Acceptable Range | Interpretation |
|--------|------------------|----------------|
| **R-hat** | < 1.01 | Measures chain convergence; values near 1.0 indicate chains mixed well |
| **ESS (Effective Sample Size)** | > 400 | Number of independent samples; higher is better |
| **Divergences** | < 1% of samples | Indicates numerical instability; should be near zero |
| **BFMI (Bayesian Fraction of Missing Information)** | > 0.3 | Low values suggest reparameterization needed |

**Troubleshooting poor diagnostics:**
- High R-hat (>1.01): Increase `num_warmup` or `num_chains`
- Low ESS (<400): Increase `num_samples` or check for strong correlations
- Many divergences: Increase `target_accept` (default 0.8) or use NLSQ warm-start


## Theory: Sticky Rouse Model

The Sticky Rouse model describes polymers with transient physical crosslinks (stickers):

**Key Physics:**
- Rouse chain has N modes with relaxation times τ_R,k
- Physical crosslinks (stickers) have lifetime τ_s
- Effective relaxation time: **τ_eff,k = max(τ_R,k, τ_s)**

**Regime Analysis:**
- When τ_s > τ_R,k: Sticker dominates → all modes relax on τ_s (sticker plateau)
- When τ_R,k > τ_s: Rouse dynamics dominate → mode-specific relaxation

**Steady-State Viscosity:**
$$\sigma(\dot{\gamma}) = \sum_k \frac{G_k \tau_{\textrm{eff},k} \dot{\gamma}}{1 + (\tau_{\textrm{eff},k} \dot{\gamma})^2} + \eta_s \dot{\gamma}$$

**Parameters (n_modes=3):**
- G_0, τ_R,0: First mode modulus and Rouse time
- G_1, τ_R,1: Second mode
- G_2, τ_R,2: Third mode
- τ_s: Sticker lifetime (critical parameter)
- η_s: Solvent viscosity

## Load Data

In [ ]:
# Load ARES flow curve data
gamma_dot, stress = load_ml_ikh_flow_curve("ARES_up")

print(f"Data shape: {len(gamma_dot)} points")
print(f"Shear rate range: {gamma_dot.min():.2e} - {gamma_dot.max():.2e} s^-1")
print(f"Stress range: {stress.min():.2e} - {stress.max():.2e} Pa")

# Plot raw data
fig, ax = plt.subplots(figsize=(8, 6))
ax.loglog(gamma_dot, stress, 'ko', label='ARES data', markersize=8)
ax.set_xlabel('Shear Rate (s$^{-1}$)', fontsize=12)
ax.set_ylabel('Stress (Pa)', fontsize=12)
ax.set_title('Flow Curve: Carbopol Gel', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.close("all")
plt.close('all')

print("\n⚠️  Note: Carbopol is not an ionomer. This demonstrates the Sticky Rouse workflow and sticker-mode analysis.")

## NLSQ Fitting

In [ ]:
# Initialize model with 3 modes
model = TNTStickyRouse(n_modes=3)
param_names = get_tnt_sticky_rouse_param_names(n_modes=3)
print(f"Model parameters ({len(param_names)}): {param_names}")

# Fit using NLSQ
print("\nFitting with NLSQ...")
start_time = time.time()
model.fit(gamma_dot, stress, test_mode="flow_curve", method='scipy')
fit_time = time.time() - start_time

# Compute metrics
stress_pred_train = model.predict(gamma_dot, test_mode="flow_curve")
metrics_nlsq = compute_fit_quality(stress, stress_pred_train)

print(f"\nFit completed in {fit_time:.2f} seconds")
print(f"R² = {metrics_nlsq['R2']:.6f}")
print(f"RMSE = {metrics_nlsq['RMSE']:.4e} Pa")

## Fitted Parameters

In [ ]:
# Extract fitted parameters
params_nlsq = {name: model.parameters.get_value(name) for name in param_names}

print("\nFitted Parameters:")
print("-" * 50)
for name, value in params_nlsq.items():
    if 'tau' in name:
        print(f"{name:10s} = {value:12.4e} s")
    elif 'eta' in name:
        print(f"{name:10s} = {value:12.4e} Pa·s")
    else:
        print(f"{name:10s} = {value:12.4e} Pa")

# Compute effective relaxation times
tau_s = params_nlsq['tau_s']
print(f"\nSticker lifetime: τ_s = {tau_s:.4e} s")
print("\nEffective Relaxation Times:")
print("-" * 50)
for i in range(3):
    tau_R = params_nlsq[f'tau_R_{i}']
    tau_eff = max(tau_R, tau_s)
    regime = "STICKER-DOMINATED" if tau_s > tau_R else "ROUSE-DOMINATED"
    print(f"Mode {i}: τ_R = {tau_R:.4e} s, τ_eff = {tau_eff:.4e} s ({regime})")

## Physical Analysis: Effective Relaxation Times

In [ ]:
gamma_dot_fine = np.logspace(np.log10(gamma_dot.min()), np.log10(gamma_dot.max()), 200)
stress_pred = model.predict(gamma_dot_fine, test_mode='flow_curve')
# Visualize sticker-mode interaction
fig = plot_sticky_rouse_effective_times(model)
plt.close("all")
plt.close('all')

## NLSQ Prediction vs Data

In [ ]:
# Compute metrics for plot title
metrics = compute_fit_quality(stress, model.predict(gamma_dot, test_mode='flow_curve'))

# Plot NLSQ fit with uncertainty band
fig, ax = plot_nlsq_fit(
    gamma_dot, stress, model, test_mode="flow_curve",
    param_names=param_names, log_scale=True,
    xlabel=r'Shear rate $\dot{\gamma}$ (s$^{-1}$)',
    ylabel=r'Shear stress $\sigma$ (Pa)',
    title=f'NLSQ Fit (R² = {metrics["R2"]:.4f})'
)
plt.close("all")

## Mode Decomposition

In [ ]:
# Plot individual mode contributions
fig = plot_mode_decomposition(model, gamma_dot_fine, "flow_curve")
plt.close("all")
plt.close('all')

## Sticker-Limited Plateau Analysis

In [ ]:
# Analyze sticker plateau regime
tau_s = params_nlsq['tau_s']
sticker_shear_rate = 1.0 / tau_s
print(f"Sticker-limited plateau region: γ̇ ~ 1/τ_s = {sticker_shear_rate:.4e} s^-1")

# Count modes in sticker-dominated regime
n_sticker_modes = sum(1 for i in range(3) if params_nlsq[f'tau_R_{i}'] < tau_s)
print(f"\nNumber of sticker-dominated modes: {n_sticker_modes}/3")

# Plateau modulus estimate (sum of sticker-dominated modes)
G_plateau = sum(params_nlsq[f'G_{i}'] for i in range(3) if params_nlsq[f'tau_R_{i}'] < tau_s)
print(f"Sticker plateau modulus: G_plateau ≈ {G_plateau:.4e} Pa")

## Bayesian Inference

In [ ]:
# FAST_MODE: Use reduced MCMC for quick validation
# FAST_MODE controls Bayesian inference (env var FAST_MODE, default=1)
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    result_bayes = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    result_bayes = model.fit_bayesian(
        gamma_dot, stress,
        test_mode='flow_curve',
        
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


## Convergence Diagnostics

In [ ]:
# Skip convergence diagnostics in CI mode
if not FAST_MODE:
    print_convergence_summary(result_bayes, param_names)
else:
    print("FAST_MODE: Skipping convergence diagnostics")


## Parameter Comparison: NLSQ vs Bayesian

In [ ]:
# Compare point estimates
print_parameter_comparison(model, result_bayes.posterior_samples, param_names)

## ArviZ Diagnostics

In [ ]:
# ArviZ diagnostics (trace, pair, forest, energy, autocorrelation, rank)
if not FAST_MODE and hasattr(result_bayes, 'to_inference_data'):
    display_arviz_diagnostics(result_bayes, param_names, fast_mode=FAST_MODE)
else:
    print("FAST_MODE: Skipping ArviZ diagnostics")

## Posterior Predictive Distribution

In [ ]:
# Posterior predictive check
if not FAST_MODE and hasattr(result_bayes, 'posterior_samples'):
    fig, ax = plot_posterior_predictive(
        gamma_dot, stress, model, result_bayes,
        test_mode="flow_curve", param_names=param_names,
        log_scale=True,
        xlabel=r'Shear rate $\\dot{\\gamma}$ (s$^{-1}$)',
        ylabel=r'Shear stress $\\sigma$ (Pa)'
    )
    plt.close("all")
else:
    print("FAST_MODE: Skipping posterior predictive")

## Physical Interpretation

In [ ]:
# Get posterior samples
posterior = result_bayes.posterior_samples

# Extract posterior means
params_bayes = {name: float(np.mean(posterior[name])) for name in param_names}
tau_s_bayes = params_bayes['tau_s']

print("Physical Interpretation (Posterior Means):")
print("=" * 60)
print(f"\nSticker Lifetime: τ_s = {tau_s_bayes:.4e} s")
print(f"Critical Shear Rate: γ̇_c ~ 1/τ_s = {1.0/tau_s_bayes:.4e} s^-1")

print("\nMode-by-Mode Analysis:")
print("-" * 60)
for i in range(3):
    G_i = params_bayes[f'G_{i}']
    tau_R_i = params_bayes[f'tau_R_{i}']
    tau_eff_i = max(tau_R_i, tau_s_bayes)
    ratio = tau_s_bayes / tau_R_i
    
    print(f"\nMode {i}:")
    print(f"  G_{i} = {G_i:.4e} Pa")
    print(f"  τ_R,{i} = {tau_R_i:.4e} s")
    print(f"  τ_eff,{i} = {tau_eff_i:.4e} s")
    print(f"  τ_s/τ_R = {ratio:.2f}")
    
    if ratio > 1.0:
        print(f"  ✓ STICKER-DOMINATED (τ_s > τ_R): Mode contributes to sticker plateau")
    else:
        print(f"  ✓ ROUSE-DOMINATED (τ_R > τ_s): Mode exhibits intrinsic Rouse dynamics")

# Estimate zero-shear viscosity
eta_0 = sum(params_bayes[f'G_{i}'] * max(params_bayes[f'tau_R_{i}'], tau_s_bayes) for i in range(3)) + params_bayes['eta_s']
print(f"\nZero-Shear Viscosity: η_0 = {eta_0:.4e} Pa·s")

# Total modulus
G_total = sum(params_bayes[f'G_{i}'] for i in range(3))
print(f"Total Modulus: G_total = {G_total:.4e} Pa")

## Save Results

In [ ]:
# Save results to disk
output_path = save_tnt_results(model, result_bayes, "sticky_rouse", "flow_curve", param_names)
print(f"Results saved to: {output_path}")

## Further Reading

### TNT Documentation

- **[TNT Model Family Overview](../../docs/source/models/tnt/index.rst)**: Complete guide to all 5 TNT models
- **[TNT Protocols Reference](../../docs/source/models/tnt/tnt_protocols.rst)**: Mathematical framework for all protocols
- **[TNT Knowledge Extraction](../../docs/source/models/tnt/tnt_knowledge_extraction.rst)**: Guide for interpreting fitted parameters

### Related Notebooks

Explore other protocols in this model family and compare with advanced TNT models.

## Next Steps

- **Notebook 26**: Startup shear with stress overshoot in Sticky Rouse
- **Notebook 27**: Stress relaxation showing hierarchical dynamics
- **Notebook 29**: SAOS with Rouse scaling regime
- **Advanced Models**: Compare with other TNT variants (Notebooks 01-24)

### Key References

1. **Tanaka, F., & Edwards, S. F.** (1992). Viscoelastic properties of physically crosslinked networks. 1. Transient network theory. *Macromolecules*, 25(5), 1516-1523. [DOI: 10.1021/ma00031a024](https://doi.org/10.1021/ma00031a024)
   - **Original TNT framework**: Conformation tensor dynamics for reversible networks

2. **Green, M. S., & Tobolsky, A. V.** (1946). A new approach to the theory of relaxing polymeric media. *Journal of Chemical Physics*, 14(2), 80-92. [DOI: 10.1063/1.1724109](https://doi.org/10.1063/1.1724109)
   - **Transient network foundation**: Network strand creation and breakage kinetics

3. **Yamamoto, M.** (1956). The visco-elastic properties of network structure I. General formalism. *Journal of the Physical Society of Japan*, 11(4), 413-421. [DOI: 10.1143/JPSJ.11.413](https://doi.org/10.1143/JPSJ.11.413)
   - **Network viscoelasticity theory**: Mathematical formulation of temporary networks

4. **Bell, G. I.** (1978). Models for the specific adhesion of cells to cells. *Science*, 200(4342), 618-627. [DOI: 10.1126/science.347575](https://doi.org/10.1126/science.347575)
   - **Bell breakage model**: Stress-dependent bond dissociation kinetics

5. **Sprakel, J., Spruijt, E., Cohen Stuart, M. A., van der Gucht, J., & Besseling, N. A. M.** (2008). Universal route to a state of pure shear flow. *Physical Review Letters*, 101(24), 248304. [DOI: 10.1103/PhysRevLett.101.248304](https://doi.org/10.1103/PhysRevLett.101.248304)
   - **TNT experimental validation**: Flow curve measurements and rheological signatures
